In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import utils
import algo
import os
import itertools
import pickle
from tqdm import tqdm
from scipy.stats import binomtest, binom
%matplotlib widget

In [ ]:
def get_features(feats_path_folder, video_id, len_seg, offset=None, smooth=True):
    with open(feats_path_folder + video_id + '_mask.pkl', 'rb') as f:
        feats = pickle.load(f)
    feats = np.concatenate(tuple(feats), axis=0)
    feats = utils.clean_features(feats, smooth=smooth)
    if offset is not None:
        end_idx = min(offset + len_seg, feats.shape[0])
        start_idx = end_idx - len_seg
        feats = feats[start_idx:end_idx, :]
    else:
        feats = feats[:len_seg, :]
    return feats

In [ ]:
def get_gaze(gaze_path, len_seg, offset=None):
    gaze = np.load(gaze_path, allow_pickle=True)
    # interpolate missing values
    gaze = np.array([np.nan if x is None else x for x in gaze])
    gaze_clean = utils.clean_features(gaze.astype(np.float64), smooth=False)
    if offset is not None:
        end_idx = min(offset + len_seg, gaze_clean.shape[0])
        start_idx = end_idx - len_seg
        gaze_clean = gaze_clean[start_idx:end_idx, :]
    else:
        gaze_clean = gaze_clean[:len_seg, :]
    return gaze_clean

In [ ]:
def get_eeg_eog(eeg_path, fsStim, bads, expdim=True):
    eeg_prepro, fs, _ = utils.preprocessing(eeg_path, HP_cutoff = 0.5, AC_freqs=50, band=None, resamp_freqs=fsStim, bads=bads, eog=True, regression=True, normalize=True)
    eeg_channel_indices = mne.pick_types(eeg_prepro.info, eeg=True)
    eog_channel_indices = mne.pick_types(eeg_prepro.info, eog=True)
    eeg_downsampled, _ = eeg_prepro[eeg_channel_indices]
    eog_downsampled, _ = eeg_prepro[eog_channel_indices]
    if expdim:
        eeg_downsampled = np.expand_dims(eeg_downsampled.T, axis=2)
        eog_downsampled = np.expand_dims(eog_downsampled.T, axis=2)
    return eeg_downsampled, eog_downsampled, fs

In [ ]:
def data_per_subj(eeg_folder, fsStim, bads, singleobj, feats_path_folder=None, expdim=True):
    eeg_files_all = [file for file in os.listdir(eeg_folder) if file.endswith('.set')]
    if singleobj:
        files = [file for file in eeg_files_all if len(file.split('_')) == 1]
    else:
        files = [file for file in eeg_files_all if len(file.split('_')) == 3]
    files.sort()
    nb_files = len(files)
    eeg_list = []
    eog_list = []
    len_seg_list = []
    gaze_list = []
    for file in files:
        eeg_downsampled, eog_downsampled, fs = get_eeg_eog(eeg_folder + file, fsStim, bads, expdim)
        eeg_list.append(eeg_downsampled)
        eog_list.append(eog_downsampled)
        len_seg_list.append(eeg_downsampled.shape[0])
        id_att = file[:-4].split('_')[-1]
        gaze_file = [file for file in os.listdir(eeg_folder) if file.endswith('.npy') and file.split('_')[-2]==id_att]
        if len(gaze_file) == 1:
            offset = 122 * fsStim if not singleobj else None
            gaze = get_gaze(eeg_folder + gaze_file[0], len_seg_list[-1], offset)
            gaze = np.expand_dims(gaze, axis=2)
        else:
            gaze = np.zeros((len_seg_list[-1], 2, 1))
        gaze_list.append(gaze)
    if feats_path_folder is not None:
        feat_att_list = []
        feat_unatt_list = []
        for i in range(len(files)):
            file = files[i]
            len_seg = len_seg_list[i]
            name = file[:-4]
            id_att = name.split('_')[-1]
            if singleobj:
                feats_att = get_features(feats_path_folder, id_att, len_seg, offset=None, smooth=True)
                feats_unatt = None
            else:
                offset = 122 * fsStim
                ids = set(name.split('_'))
                ids.remove(id_att)
                id_unatt = ids.pop()
                feats_att = get_features(feats_path_folder, id_att, len_seg, offset, smooth=True)
                feats_unatt = get_features(feats_path_folder, id_unatt, len_seg, offset, smooth=True)
            feat_att_list.append(feats_att)
            feat_unatt_list.append(feats_unatt)
    else:
        feat_att_list = None
        feat_unatt_list = None
    return eeg_list, eog_list, feat_att_list, feat_unatt_list, gaze_list, fs, nb_files, len_seg_list

In [ ]:
def data_multi_subj(subj_path, fsStim, bads, singleobj, feats_path_folder, SAVE=True):
    PATTERN = subj_path[0].split('/')[-3]
    data_path = 'data/' + PATTERN + '/'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    nb_subj = len(subj_path)
    eeg_multisubj_list, eog_multisubj_list, feat_att_list, feat_unatt_list, gaze_multisubj_list, fs, nb_files, len_seg_list = data_per_subj(subj_path[0], fsStim, bads[0], singleobj, feats_path_folder)
    for n in range(1,nb_subj):
        eeg_list, eog_list, _, _, gaze_list, _, nb_files_sub, _ = data_per_subj(subj_path[n], fsStim, bads[n], singleobj, feats_path_folder=None)
        assert nb_files == nb_files_sub
        eeg_multisubj_list = [np.concatenate((eeg_multisubj_list[i], eeg_list[i]), axis=2) for i in range(nb_files)]
        eog_multisubj_list = [np.concatenate((eog_multisubj_list[i], eog_list[i]), axis=2) for i in range(nb_files)]
        gaze_multisubj_list = [np.concatenate((gaze_multisubj_list[i], gaze_list[i]), axis=2) for i in range(nb_files)]
    if SAVE:
        # save all data (eeg_multisubj_list, eog_multisubj_list, feat_att_list, feat_unatt_list, fs, nb_files) into a single file
        data = {'eeg_multisubj_list': eeg_multisubj_list, 'eog_multisubj_list': eog_multisubj_list, 'feat_att_list': feat_att_list, 'feat_unatt_list': feat_unatt_list, 'gaze_multisubj_list': gaze_multisubj_list, 'fs': fs, 'len_seg_list': len_seg_list}
        file_name = 'data_singleobj.pkl' if singleobj else 'data_twoobj.pkl'
        with open(data_path + file_name, 'wb') as f:
            pickle.dump(data, f)
    return eeg_multisubj_list, eog_multisubj_list, feat_att_list, feat_unatt_list, gaze_multisubj_list, fs, len_seg_list

In [ ]:
def add_new_data(subj_path, fsStim, bads, feats_path_folder, singleobj):
    PATTERN = subj_path[0].split('/')[-3]
    data_path = 'data/' + PATTERN + '/'
    file_name = 'data_singleobj.pkl' if singleobj else 'data_twoobj.pkl'
    with open(data_path + file_name, 'rb') as f:
        data = pickle.load(f)
    nb_subj_old = data['eeg_multisubj_list'][0].shape[2]
    eeg_multisubj_add, eog_multisubj_add, _, _, gaze_multisubj_add, _, _ = data_multi_subj(subj_path[nb_subj_old:], fsStim, bads[nb_subj_old:], singleobj, feats_path_folder, SAVE=False)
    eeg_multisubj_list = [np.concatenate((old, new), axis=2) for old, new in zip(data['eeg_multisubj_list'], eeg_multisubj_add)]
    eog_multisubj_list = [np.concatenate((old, new), axis=2) for old, new in zip(data['eog_multisubj_list'], eog_multisubj_add)]
    gaze_multisubj_list = [np.concatenate((old, new), axis=2) for old, new in zip(data['gaze_multisubj_list'], gaze_multisubj_add)]
    data['eeg_multisubj_list'] = eeg_multisubj_list
    data['eog_multisubj_list'] = eog_multisubj_list
    data['gaze_multisubj_list'] = gaze_multisubj_list
    with open(data_path + file_name, 'wb') as f:
        pickle.dump(data, f)
    return eeg_multisubj_list, eog_multisubj_list, data['feat_att_list'], data['feat_unatt_list'], gaze_multisubj_list, data['fs'], data['len_seg_list']

In [ ]:
def remove_shot_cuts(data, fs, time_points=None, remove_time=1):
    T = data.shape[0]
    if time_points is None:
        time_points = [0, T]
    nearby_idx = []
    for p in time_points:
        len_points = int(remove_time*fs)
        nearby_idx = nearby_idx + list(range(max(0, p-len_points), min(p+len_points, T)))
    nearby_idx = list(set(nearby_idx))
    data_clean = np.delete(data, nearby_idx, axis=0)
    return data_clean

In [ ]:
def load_data(subj_path, fsStim, bads, feats_path_folder, PATTERN, singleobj, LOAD_ONLY, ALL_NEW):
    file_name = 'data_singleobj.pkl' if singleobj else 'data_twoobj.pkl'
    if LOAD_ONLY:
        data_path = 'data/' + PATTERN + '/'
        with open(data_path + file_name, 'rb') as f:
            data = pickle.load(f)
        eeg_multisubj_list = data['eeg_multisubj_list']
        eog_multisubj_list = data['eog_multisubj_list']
        feat_att_list = data['feat_att_list']
        feat_unatt_list = data['feat_unatt_list']
        gaze_multisubj_list = data['gaze_multisubj_list']
        fs = data['fs']
        len_seg_list = data['len_seg_list']
    else:
        if ALL_NEW:
            eeg_multisubj_list, eog_multisubj_list, feat_att_list, feat_unatt_list, gaze_multisubj_list, fs, len_seg_list = data_multi_subj(subj_path, fsStim, bads, singleobj, feats_path_folder)
        else:
            eeg_multisubj_list, eog_multisubj_list, feat_att_list, feat_unatt_list, gaze_multisubj_list, fs, len_seg_list = add_new_data(subj_path, fsStim, bads, feats_path_folder, singleobj)
    return eeg_multisubj_list, eog_multisubj_list, feat_att_list, feat_unatt_list, gaze_multisubj_list, fs, len_seg_list

In [ ]:
# Check the alignment between eog and gaze. The synchronization is good if the peaks of two signals (eye blinks) are aligned.
def check_alignment(subj_ID, eog_multisubj_list, gaze_multisubj_list, nb_points=500):
    eog_one_subj_list = [eog[:,:,subj_ID] for eog in eog_multisubj_list]
    gaze_one_subj_list = [gaze[:,:,subj_ID] for gaze in gaze_multisubj_list]
    eog_verti_list = [eog[:,0] - eog[:,1] for eog in eog_one_subj_list]
    gaze_y_list = [gaze[:,1] for gaze in gaze_one_subj_list]
    nb_videos = len(eog_verti_list)
    # make a subplot (3 x (nb_videos//3+1)) for each video
    plt.close()
    nb_rows = 3
    nb_cols = nb_videos//3+1
    fig, ax = plt.subplots(nb_rows, nb_cols, figsize=(15, 10))
    for i in range(nb_videos):
        ax[i//nb_cols, i%nb_cols].plot(eog_verti_list[i][-nb_points:]/np.max(eog_verti_list[i][-nb_points:]), label='eog vertical')
        ax[i//nb_cols, i%nb_cols].plot(gaze_y_list[i][-nb_points:]/np.max(gaze_y_list[i][-nb_points:]), label='gaze y')
        ax[i//nb_cols, i%nb_cols].set_title('Video ' + str(i+1))
        ax[i//nb_cols, i%nb_cols].legend()
    plt.show()

### Load data

In [ ]:
subjects = ['Pilot_1', 'Pilot_2', 'Pilot_4', 'Pilot_5']
PATTERN = 'Overlay'
subj_path = ['../../Experiments/data/Two_Obj/' + PATTERN + '/' + sub + '/' for sub in subjects]
nb_subj = len(subjects)
bads = [['A30', 'B25'], ['B25'], ['B25'], []] 
fsStim = 30
feats_path_folder = '../Feat_Multi/features/'

In [ ]:
%%capture
singleobj = False
eeg_multisubj_list, eog_multisubj_list, feat_all_att_list, feat_all_unatt_list, gaze_multisubj_list, fs, len_seg_list = load_data(subj_path, fsStim, bads, feats_path_folder, PATTERN, singleobj, LOAD_ONLY=False, ALL_NEW=False)

In [ ]:
# Check the alignment between eog and gaze. The synchronization is good if the peaks of two signals (eye blinks) are aligned.
subj_to_check = 'Pilot_5'
subj_ID = subjects.index(subj_to_check)
check_alignment(subj_ID, eog_multisubj_list, gaze_multisubj_list, nb_points=500)

In [ ]:
RemoveSC = True
if RemoveSC:
    eeg_multisubj_list = [remove_shot_cuts(eeg, fs) for eeg in eeg_multisubj_list]
    eog_multisubj_list = [remove_shot_cuts(eog, fs) for eog in eog_multisubj_list]
    gaze_multisubj_list = [remove_shot_cuts(gaze, fs) for gaze in gaze_multisubj_list]
    feat_all_att_list = [remove_shot_cuts(feat, fs) for feat in feat_all_att_list]
    if not singleobj:
        feat_all_unatt_list = [remove_shot_cuts(feat, fs) for feat in feat_all_unatt_list]

In [ ]:
# eeg_band_list = [utils.extract_freq_band(eeg, fsStim, band=[0.1,5]) for eeg in eeg_multisubj_list]

In [ ]:
objflow_att_list = [feats[:,8] for feats in feat_all_att_list]
objtempctr_att_list = [feats[:,17] for feats in feat_all_att_list]
if not singleobj:
    objflow_unatt_list = [feats[:,8] for feats in feat_all_unatt_list]
    objtempctr_unatt_list = [feats[:,17] for feats in feat_all_unatt_list]

In [ ]:
objflow_att = np.concatenate(objflow_att_list, axis=0)
objtempctr_att = np.concatenate(objtempctr_att_list, axis=0)
if not singleobj:
    objflow_unatt = np.concatenate(objflow_unatt_list, axis=0)
    objtempctr_unatt = np.concatenate(objtempctr_unatt_list, axis=0)
    # calculate the correlation between objflow_att and objflow_unatt
    print(np.corrcoef(objflow_att, objflow_unatt)[0,1])
    # calculate the correlation between objtempctr_att and objtempctr_unatt
    print(np.corrcoef(objtempctr_att, objtempctr_unatt)[0,1])

In [ ]:
L_EEG = 3 
L_Stim = int(fsStim/2) 
offset_EEG = 1 
offset_Stim = 0 
trial_len_list = list(range(5, 125, 5))

In [ ]:
figure_path = 'figures/' + PATTERN + '/SingleObj/' if singleobj else 'figures/' + PATTERN + '/TwoObj/'
if not os.path.exists(figure_path):
    os.makedirs(figure_path)
table_path = 'tables/' + PATTERN + '/SingleObj/' if singleobj else 'tables/' + PATTERN + '/TwoObj/'
if not os.path.exists(table_path):
    os.makedirs(table_path)

### Mode=Compete: Discriminating between attended and unattended segments

In [ ]:
def pipe_att_or_unatt(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT, figure_dir=None, trial_len=60, fold=5, n_components=5, nb_comp_into_account=2, PLOT=False, signifi_level=True, message=True):
    eeg_onesubj_list = [eeg[:,:,Subj_ID] for eeg in eeg_multisubj_list]
    CCA = algo.CanonicalCorrelationAnalysis(eeg_onesubj_list, feat_att_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, fold=fold, n_components=n_components, signifi_level=signifi_level, message=message)
    corr_att_fold, corr_unatt_fold, V_eeg_train, V_feat_train, sig_corr_att, sig_corr_unatt = CCA.att_or_unatt(feat_unatt_list, trial_len=trial_len, TRAIN_WITH_ATT=TRAIN_WITH_ATT)
    acc, p_value, acc_sig, corr_att_cv, corr_unatt_cv= utils.eval_compete(corr_att_fold, corr_unatt_fold, nb_comp_into_account)
    if PLOT:
        # find the indices components with corr > sig_corr
        idx_sig_att = np.where(corr_att_cv > sig_corr_att)[0]
        idx_sig_unatt = np.where(corr_unatt_cv > sig_corr_unatt)[0]
        figure_name_att = figure_dir + 'Subj_' + str(Subj_ID) + '_Trial_len_' + str(trial_len) +  '_Train_Att_Test_Att.png' if TRAIN_WITH_ATT else figure_dir + 'Subj_' + str(Subj_ID) + '_Trial_len_' + str(trial_len) + '_Train_Unatt_Test_Att.png'
        figure_name_unatt = figure_dir + 'Subj_' + str(Subj_ID) + '_Trial_len_' + str(trial_len) + '_Train_Att_Test_Unatt.png' if TRAIN_WITH_ATT else figure_dir + 'Subj_' + str(Subj_ID) + '_Trial_len_' + str(trial_len) + '_Train_Unatt_Test_Unatt.png'
        eeg_onesub = np.concatenate(tuple(eeg_onesubj_list), axis=0)
        forward_model = CCA.forward_model(eeg_onesub, V_eeg_train)
        utils.plot_spatial_resp(forward_model, corr_att_fold, figure_name_att, idx_sig=idx_sig_att)
        utils.plot_spatial_resp(forward_model, corr_unatt_fold, figure_name_unatt, idx_sig=idx_sig_unatt)
    return acc, p_value, acc_sig, corr_att_cv, corr_unatt_cv

In [ ]:
def pipe_compete_trials(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dir, fold=5, n_components=5, nb_comp_into_account=2):
    acc_list = []
    p_value_list = []
    acc_sig_list = []
    for trial_len in trial_len_list:
        print('Trial length: ', trial_len)
        acc, p_value, acc_sig, _, _ = pipe_att_or_unatt(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, trial_len=trial_len, fold=fold, n_components=n_components, nb_comp_into_account=nb_comp_into_account, signifi_level=False, message=False)
        acc_list.append(acc)
        p_value_list.append(p_value)
        acc_sig_list.append(acc_sig)
        # save the results
    results = {'acc': acc_list, 'pvalue': p_value_list, 'acc_sig': acc_sig_list, 'trial_len_list': trial_len_list}
    table_name = table_dir + 'Subj_' + str(Subj_ID) + '_compete.pkl'
    with open(table_name, 'wb') as f:
        pickle.dump(results, f)

In [ ]:
feat_att_list = objflow_att_list
feat_unatt_list = objflow_unatt_list
figure_dir_EEG = figure_path + '/EEG/'
if not os.path.exists(figure_dir_EEG):
    os.makedirs(figure_dir_EEG)
table_dir_EEG = table_path + '/EEG/'
if not os.path.exists(table_dir_EEG):
    os.makedirs(table_dir_EEG)

In [ ]:
Subj_ID = 3
acc, p_value, acc_sig, corr_att_cv, corr_unatt_cv= pipe_att_or_unatt(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, figure_dir=figure_dir_EEG, trial_len=60, PLOT=True)
acc, p_value, acc_sig, corr_att_cv, corr_unatt_cv = pipe_att_or_unatt(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=False, figure_dir=figure_dir_EEG, trial_len=60, PLOT=True)

In [ ]:
pipe_compete_trials(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dir_EEG)

## Mode=Match-Mismatch: Discriminating between match and mismatch segments

In [ ]:
def pipe_mm_trials(Subj_ID, eeg_multisubj_list, feat_match_list, feat_distract_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dir, MATCHATT, fold=5, n_components=5, nb_comp_into_account=2):
    acc_list = []
    p_value_list = []
    eeg_onesubj_list = [eeg[:,:,Subj_ID] for eeg in eeg_multisubj_list]
    for trial_len in trial_len_list:
        print('Trial length: ', trial_len)
        CCA = algo.CanonicalCorrelationAnalysis(eeg_onesubj_list, feat_match_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, fold=fold, n_components=n_components, signifi_level=False, message=False)
        corr_tensor_list = CCA.match_mismatch(trial_len, feat_distract_list)
        acc, p_value = utils.eval_mm(corr_tensor_list, nb_comp_into_account)
        print('Accuracy: ', acc, 'P-value: ', p_value)
        acc_list.append(acc)
        p_value_list.append(p_value)
        # save the results
    results = {'acc': acc_list, 'pvalue': p_value_list, 'trial_len_list': trial_len_list}
    table_name = table_dir + 'Subj_' + str(Subj_ID) + '_match_att.pkl' if MATCHATT else table_dir + 'Subj_' + str(Subj_ID) + '_match_unatt.pkl'
    with open(table_name, 'wb') as f:
        pickle.dump(results, f)

In [ ]:
table_dir_EEG = table_path + '/EEG/'
if not os.path.exists(table_dir_EEG):
    os.makedirs(table_dir_EEG)

In [ ]:
Subj_ID = 3
feat_match_list = objflow_att_list
feat_distract_list = objflow_unatt_list
MATCHATT = True
pipe_mm_trials(Subj_ID, eeg_multisubj_list, feat_match_list, feat_distract_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dir_EEG, MATCHATT, fold=5, n_components=5, nb_comp_into_account=2)

In [ ]:
Subj_ID = 3
feat_match_list = objflow_unatt_list
feat_distract_list = objflow_att_list
MATCHATT = False
pipe_mm_trials(Subj_ID, eeg_multisubj_list, feat_match_list, feat_distract_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dir_EEG, MATCHATT, fold=5, n_components=5, nb_comp_into_account=2)

In [ ]:
# load results
Subj_ID = 3
with open(table_dir_EEG + 'Subj_' + str(Subj_ID) + '_compete.pkl', 'rb') as f:
    results = pickle.load(f)
    acc_list_compete = results['acc']
    p_values_list_compete = results['pvalue']
    acc_sig_list = results['acc_sig']
    trial_len_list = results['trial_len_list']
with open(table_dir_EEG + 'Subj_' + str(Subj_ID) + '_match_att.pkl', 'rb') as f:
    results = pickle.load(f)
    acc_list_match_att = results['acc']
    p_values_list_match_att = results['pvalue']
with open(table_dir_EEG + 'Subj_' + str(Subj_ID) + '_match_unatt.pkl', 'rb') as f:
    results = pickle.load(f)
    acc_list_match_unatt = results['acc']
    p_values_list_match_unatt = results['pvalue']

plt.close()
# Plot lines
plt.plot(trial_len_list, acc_list_compete, label='Competing')
plt.plot(trial_len_list, acc_list_match_att, label='Matching-Att')
plt.plot(trial_len_list, acc_list_match_unatt, label='Matching-Unatt')
plt.plot(trial_len_list, acc_sig_list, label='Significance level', linestyle='--')

# Assuming p_values_list, p_values_list_unatt_permu, and p_values_list_permu_att are the lists of p-values for each condition
# Plot circles where p-value < 0.05
for trial_len, acc, p_value in zip(trial_len_list, acc_list_compete, p_values_list_compete):
    if p_value > 0.05:
        plt.scatter(trial_len, acc, edgecolors='red', facecolors='none', s=100)

for trial_len, acc, p_value in zip(trial_len_list, acc_list_match_att, p_values_list_match_att):
    if p_value > 0.05:
        plt.scatter(trial_len, acc, edgecolors='red', facecolors='none', s=100)

for trial_len, acc, p_value in zip(trial_len_list, acc_list_match_unatt, p_values_list_match_unatt):
    if p_value > 0.05:
        plt.scatter(trial_len, acc, edgecolors='red', facecolors='none', s=100)

plt.legend()
plt.xlabel('Segment length (s)')
plt.ylabel('Accuracy')
plt.title('Subject ' + str(Subj_ID+1))
plt.show()

## Using EOG signals or eye tracking data

In [ ]:
feat_att_list = objflow_att_list
feat_unatt_list = objflow_unatt_list
figure_dir_EOG= figure_path + '/EOG/'
if not os.path.exists(figure_dir_EOG):
    os.makedirs(figure_dir_EOG)
table_dir_EOG = table_path + '/EOG/'
if not os.path.exists(table_dir_EOG):
    os.makedirs(table_dir_EOG)

In [ ]:
Subj_ID = 3
acc, p_value, acc_sig, corr_att_cv, corr_unatt_cv= pipe_att_or_unatt(Subj_ID, eog_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, figure_dir=figure_dir_EOG, trial_len=60, PLOT=False)
acc, p_value, acc_sig, corr_att_cv, corr_unatt_cv = pipe_att_or_unatt(Subj_ID, eog_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=False, figure_dir=figure_dir_EOG, trial_len=60, PLOT=False)

In [ ]:
pipe_compete_trials(Subj_ID, eog_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dir_EOG)

In [ ]:
feat_att_list = objflow_att_list
feat_unatt_list = objflow_unatt_list
figure_dir_ET= figure_path + '/ET/'
if not os.path.exists(figure_dir_ET):
    os.makedirs(figure_dir_ET)
table_dir_ET = table_path + '/ET/'
if not os.path.exists(table_dir_ET):
    os.makedirs(table_dir_ET)

In [ ]:
Subj_ID = 3
acc, p_value, acc_sig, corr_att_cv, corr_unatt_cv= pipe_att_or_unatt(Subj_ID, gaze_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, figure_dir=figure_dir_ET, trial_len=60, PLOT=False)
acc, p_value, acc_sig, corr_att_cv, corr_unatt_cv = pipe_att_or_unatt(Subj_ID, gaze_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=False, figure_dir=figure_dir_ET, trial_len=60, PLOT=False)

In [ ]:
pipe_compete_trials(Subj_ID, gaze_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dir_ET)

In [ ]:
# load results
Subj_ID = 3
with open(table_dir_EEG + 'Subj_' + str(Subj_ID) + '_compete.pkl', 'rb') as f:
    results = pickle.load(f)
    acc_list_EEG = results['acc']
    p_values_list_EEG = results['pvalue']
    acc_sig_list = results['acc_sig']
    trial_len_list = results['trial_len_list']
with open(table_dir_EOG + 'Subj_' + str(Subj_ID) + '_compete.pkl', 'rb') as f:
    results = pickle.load(f)
    acc_list_EOG = results['acc']
    p_values_list_EOG = results['pvalue']
with open(table_dir_ET + 'Subj_' + str(Subj_ID) + '_compete.pkl', 'rb') as f:
    results = pickle.load(f)
    acc_list_ET = results['acc']
    p_values_list_ET = results['pvalue']


plt.close()
# Plot lines
plt.plot(trial_len_list, acc_list_EEG, label='EEG')
plt.plot(trial_len_list, acc_list_EOG, label='EOG')
plt.plot(trial_len_list, acc_list_ET, label='Eye tracker')
plt.plot(trial_len_list, acc_sig_list, label='Significance level', linestyle='--')

# Assuming p_values_list, p_values_list_unatt_permu, and p_values_list_permu_att are the lists of p-values for each condition
# Plot circles where p-value < 0.05
for trial_len, acc, p_value in zip(trial_len_list, acc_list_EEG, p_values_list_EEG):
    if p_value > 0.05:
        plt.scatter(trial_len, acc, edgecolors='red', facecolors='none', s=100)

for trial_len, acc, p_value in zip(trial_len_list, acc_list_EOG, p_values_list_EOG):
    if p_value > 0.05:
        plt.scatter(trial_len, acc, edgecolors='red', facecolors='none', s=100)

for trial_len, acc, p_value in zip(trial_len_list, acc_list_ET, p_values_list_ET):
    if p_value > 0.05:
        plt.scatter(trial_len, acc, edgecolors='red', facecolors='none', s=100)
plt.legend()
plt.xlabel('Segment length (s)')
plt.ylabel('Accuracy')
plt.title('Subject ' + str(Subj_ID+1))
plt.show()

## Correlations between EEG signals when looking at the same video pairs but attending different objects

In [ ]:
L_eeg = 3
offset_eeg = 1
eeg_set1_onesubj_list = [eeg[:,:,Subj_ID] for eeg in eeg_set1_list]
eeg_set2_onesubj_list = [eeg[:,:,Subj_ID] for eeg in eeg_set2_list]
CCA = algo.CanonicalCorrelationAnalysis(eeg_set1_onesubj_list, eeg_set2_onesubj_list, fsStim, L_eeg, L_eeg, offset_eeg, offset_eeg, fold=5, signifi_level=True, n_components=5)
_, corr_test, sig_corr, _, tsc_test, _, _, V_A_train, V_B_train = CCA.cross_val()

In [ ]:
# Concatenate the EEG signals of different subjects together (along the time axis)
def concat_multi_subj_eeg(eeg_multisubj):
    _, _, nb_subj = eeg_multisubj.shape
    eeg_subj_list = [eeg_multisubj[:,:,i] for i in range(nb_subj)]
    eeg_concat = np.concatenate(tuple(eeg_subj_list), axis=0)
    return eeg_concat

In [ ]:
eeg_set1_array_list = [concat_multi_subj_eeg(eeg) for eeg in eeg_set1_list]
eeg_set2_array_list = [concat_multi_subj_eeg(eeg) for eeg in eeg_set2_list]

In [ ]:
CCA = algo.CanonicalCorrelationAnalysis(eeg_set1_array_list, eeg_set2_array_list, fsStim, L_eeg, L_eeg, offset_eeg, offset_eeg, fold=5, signifi_level=True, n_components=5)
_, corr_test, sig_corr, _, tsc_test, _, _, V_A_train, V_B_train = CCA.cross_val()

In [ ]:
eeg_set1_set2_tensor_list = [np.concatenate((eeg_set1, eeg_set2), axis=2) for eeg_set1, eeg_set2 in zip(eeg_set1_list, eeg_set2_list)]

In [ ]:
GCCA = algo.GeneralizedCCA(eeg_set1_set2_tensor_list, fs, L=5, offset=2, n_components=10, signifi_level=True, trials=True)
corr_train, corr_test, cov_train, cov_test, tsc_train, tsc_test, dist_train, dist_test, W_train, F_train_GCCA = GCCA.cross_val()

In [ ]:
utils.plot_spatial_resp(F_train_GCCA, corr_test, 'set1_set2.pdf', ifISC=True)